# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [5]:
# Define the target set y using the credit_ranking column

y = loans_df['credit_ranking']

# Display a sample of y

y

0       0
1       0
2       0
3       1
4       0
       ..
1594    0
1595    1
1596    1
1597    0
1598    1
Name: credit_ranking, Length: 1599, dtype: int64

In [6]:
# Define features set X by selecting all columns but credit_ranking

X = loans_df.drop(columns='credit_ranking')

# Review the features DataFrame

X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [7]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [8]:
# Create a StandardScaler instance

X_scaler = StandardScaler()

# Fit the scaler to the features training dataset

X_scaler.fit(X_train)

# Fit the scaler to the features training dataset

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [9]:
# Define the number of inputs (features) to the model
input_dim = X_train_scaled.shape[1]

# Review the number of features
print(f"Number of features: {input_dim}")

# Create the deep neural network model
model = Sequential([
    Input(shape=(input_dim,)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Assuming binary classification
])

# Display the model architecture
model.summary()


Number of features: 11


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,881 (11.25 KB)

 Trainable params: 2,881 (11.25 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# Define the number of hidden nodes for the first hidden layer
hidden1_nodes = 64

# Define the number of hidden nodes for the second hidden layer
hidden2_nodes = 32

# Define the number of neurons in the output layer
output_nodes = 1


In [11]:
# Create the Sequential model instance
model = Sequential()

# Add the first hidden layer
model.add(Dense(hidden1_nodes, activation='relu', input_dim=input_dim))

# Add the second hidden layer
model.add(Dense(hidden2_nodes, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
model.add(Dense(output_nodes, activation='sigmoid'))

# Display the model architecture
model.summary()


C:\Users\recca\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,881 (11.25 KB)

 Trainable params: 2,881 (11.25 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# Display the Sequential model summary
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,881 (11.25 KB)

 Trainable params: 2,881 (11.25 KB)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [13]:
# Compile the Sequential model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [14]:
# Fit the model using 50 epochs and the training data
history = model.fit(X_train_scaled, y_train, 
                    epochs=50, 
                    batch_size=32, 
                    validation_split=0.2)

Epoch 1/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.4945 - loss: 0.7338 - val_accuracy: 0.7042 - val_loss: 0.6164
Epoch 2/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7192 - loss: 0.5967 - val_accuracy: 0.7167 - val_loss: 0.5606
Epoch 3/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7314 - loss: 0.5453 - val_accuracy: 0.7333 - val_loss: 0.5392
Epoch 4/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7685 - loss: 0.5119 - val_accuracy: 0.7375 - val_loss: 0.5312
Epoch 5/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7729 - loss: 0.5082 - val_accuracy: 0.7375 - val_loss: 0.5291
Epoch 6/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7630 - loss: 0.4968 - val_accuracy: 0.7458 - val_loss: 0.5226
Epoch 7/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7720 - loss: 0.4931 - val_accuracy: 0.7500 - val_loss: 0.5204
Epoch 8/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7819 - loss: 0.4942 - val_accuracy: 0.7583 - val_loss:

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [15]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)

# Display the model loss and accuracy results
print(f"Loss: {test_loss}, Accuracy: {test_accuracy}")

Loss: 0.5929840207099915, Accuracy: 0.7524999976158142


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [16]:
# Set the model's file path
model_file_path = 'student_loans.keras'

# Export your model to a keras file
model.save(model_file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [17]:
# Set the model's file path
model_file_path = 'student_loans.keras'

# Load the model to a new object
loaded_model = load_model(model_file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [18]:
# Make predictions with the test data
predictions = loaded_model.predict(X_test_scaled, verbose=1)

# Display a sample of the predictions
predictions[:5]

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


array([[0.17087917],
       [0.42090854],
       [0.8233187 ],
       [0.84502697],
       [0.98151636]], dtype=float32)

In [19]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame((predictions > 0.5).astype(float), columns=['predictions'])

# Display a sample of the predictions DataFrame
print(predictions_df)

     predictions
0            0.0
1            0.0
2            1.0
3            1.0
4            1.0
..           ...
395          1.0
396          0.0
397          1.0
398          0.0
399          0.0

[400 rows x 1 columns]


### Step 4: Display a classification report with the y test data and predictions

In [20]:
# Print the classification report with the y test data and predictions
report = classification_report(y_test, predictions_df['predictions'])
print(report)

              precision    recall  f1-score   support

           0       0.73      0.76      0.74       188
           1       0.78      0.75      0.76       212

    accuracy                           0.75       400
   macro avg       0.75      0.75      0.75       400
weighted avg       0.75      0.75      0.75       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**
To build an effective recommendation system for student loan options, the following data should be collected:

Student Demographics: Age, gender, location, and socioeconomic background.
Relevance: Helps in understanding the diverse needs and eligibility criteria of different student groups.

Academic Information: Major, GPA, academic standing, and institution type (e.g., public vs. private).
Relevance: Influences loan eligibility, interest rates, and repayment options based on academic performance and field of study.

Financial Data: Current income, existing debts, credit score, and financial aid received.
Relevance: Critical for assessing loan affordability, determining interest rates, and tailoring repayment plans.

Loan History: Previous loan amounts, repayment history, and default rates.
Relevance: Provides insights into a student's reliability in managing loans, aiding in risk assessment.

Loan Preferences: Desired loan amount, preferred repayment period, and interest rate preferences.
Relevance: Ensures recommendations align with the student's financial goals and comfort levels.

Employment Information: Current employment status, job sector, and projected future earnings.
Relevance: Assists in forecasting repayment capabilities and recommending suitable loan structures.

Collecting this comprehensive data ensures that the recommendation system can provide personalized and relevant loan options tailored to each student's unique financial and academic profile.


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**
Content-Based Filtering would be the most suitable approach for this recommendation system.

Justification:

Data Utilized: The system relies heavily on individual student attributes such as demographics, academic information, financial data, and loan preferences.
Personalization: Content-based filtering leverages these specific features to recommend loan options that closely match each student's profile and needs.
Independence from Other Users: Unlike collaborative filtering, content-based methods do not depend on the behavior or preferences of other students, making it ideal when user interaction data is limited or when recommendations need to be highly personalized based on explicit attributes.
This approach ensures that each recommendation is directly aligned with the student's personal and financial circumstances, enhancing the relevance and effectiveness of the loan suggestions.

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**
**a. Data Privacy and Security

Concern: Handling sensitive personal and financial information requires stringent data protection measures to prevent breaches and misuse.
Implications: Non-compliance with data protection regulations (e.g., GDPR, FERPA) can lead to legal repercussions and loss of trust among users. Ensuring robust encryption, secure storage, and ethical data handling practices is paramount.
**b. Bias and Fairness in Recommendations

Concern: The system might inadvertently favor or disadvantage certain groups of students based on biased data or flawed algorithms.
Implications: Biased recommendations can exacerbate financial inequalities and result in unfair access to loan options. It is essential to regularly audit and refine the recommendation algorithms to ensure they promote equity and do not perpetuate existing biases.

Addressing these challenges is crucial for developing a trustworthy, equitable, and legally compliant recommendation system that genuinely serves the diverse needs of all students.
